In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd

# Load dataset
df = pd.read_csv('Conversation.csv')
questions = df['question'].tolist()
answers = df['answer'].tolist()

'''# Sample dataset of questions and answers
questions = [
    "What is your name?",
    "How are you?",
    "What do you do?",
    "Tell me a joke.",
    "What is the weather like?"
]

answers = [
    "I am a chatbot.",
    "I am doing great, thank you!",
    "I am here to chat with you.",
    "Why did the scarecrow win an award? Because he was outstanding in his field!",
    "I don't know, what's your favorite weather?"
]'''

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(questions + answers)

# Convert texts to sequences
question_sequences = tokenizer.texts_to_sequences(questions)
answer_sequences = tokenizer.texts_to_sequences(answers)

# Pad sequences
max_len = max(max(len(seq) for seq in question_sequences),max(len(seq) for seq in question_sequences))
question_padded = pad_sequences(question_sequences, maxlen=max_len, padding='post')
answer_padded = pad_sequences(answer_sequences, maxlen=max_len, padding='post')

# Prepare input and output
X = question_padded
y = answer_padded

# Model parameters
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
embedding_dim = 128
rnn_units = 32

# Build the RNN model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len),
    tf.keras.layers.SimpleRNN(vocab_size, return_sequences=True),
    tf.keras.layers.Dense(vocab_size,activation='softmax')
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X, np.expand_dims(y, -1), epochs=200, verbose=1)

# Function to predict the answer based on the question
def predict_answer(question):
    question_seq = tokenizer.texts_to_sequences([question])
    question_padded = pad_sequences(question_seq, maxlen=max_len, padding='post')
    prediction = model.predict(question_padded)
    predicted_word_index = np.argmax(prediction, axis=-1)[0]
    predicted_words = [tokenizer.index_word[i] for i in predicted_word_index if i > 0]
    return ' '.join(predicted_words)

# Test the model with a sample question
test_question = "How are you?"
response = predict_answer(test_question)
print(f"Response: {response}")

Epoch 1/200


c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


117/117 ━━━━━━━━━━━━━━━━━━━━ 102s 807ms/step - accuracy: 0.6051 - loss: 3.6858
Epoch 2/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 102s 872ms/step - accuracy: 0.6543 - loss: 2.9707
Epoch 3/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 86s 732ms/step - accuracy: 0.6582 - loss: 2.9137
Epoch 4/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 143s 737ms/step - accuracy: 0.6622 - loss: 2.8054
Epoch 5/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 86s 736ms/step - accuracy: 0.6617 - loss: 2.7597
Epoch 6/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 86s 738ms/step - accuracy: 0.6629 - loss: 2.7210
Epoch 7/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 86s 738ms/step - accuracy: 0.6562 - loss: 2.7205
Epoch 8/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 86s 738ms/step - accuracy: 0.6570 - loss: 2.6847
Epoch 9/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 86s 737ms/step - accuracy: 0.6637 - loss: 2.6206
Epoch 10/200
117/117 ━━━━━━━━━━━━━━━━━━━━ 87s 741ms/step - accuracy: 0.6627 - loss: 2.6760
Epoch 11/200
 50/117 ━━━━━━━━━━━━━━━━━━━━ 55s 821ms/step - accuracy: 0.6520 - loss: 2.7099

In [6]:
print("Chatbot is ready to chat! Type 'exit' to quit.")
while True:
    user_input = input("You: ")
    print("user:",user_input)
    if user_input.lower() == 'exit':
        print("Chatbot: Goodbye!")
        break
    response = predict_answer(user_input)
    print("Chatbot:", response)

Chatbot is ready to chat! Type 'exit' to quit.
user: hi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Chatbot: hello
user: you know tamil
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Chatbot: yes
user: your education
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Chatbot: sweets
user: your education
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Chatbot: sweets
user: your favourite food
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Chatbot: sweets
user: which is your favourite colour
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Chatbot: blue
user: exit
Chatbot: Goodbye!
